# More on DataFrames - missing values, indexing...

## Import libs

In [0]:
# import a generic pandas object and also a few specific functions that we'll use
import pandas as pd 
from google.colab import files

## Upload a file to the /content folder on google colab
* Select the file you want to upload (the csv file that I sent out)
* It will load into your 'contents' folder
* Then you can interact with it just like a normal file on your hardrive



In [0]:
%pwd
%ls

In [0]:
files.upload()

In [0]:
%ls

## Make a DataFrame object to hold the contents of the data set
[DataFrame help page](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.html)

* Just like with the pd.Series call, you can specify the data, **index** labels (row labels in this case)
* In addition to row labels, you can also specify column labels (with 'columns')
* Can also specify data type (default is inferred)
* If you read in the data from a csv file, you will be able to inheret row and column labels (if they are specified in the file). 

In [0]:
# make the call to pd.DataFrames to create the DF - usage much like pd.Series
df = pd.read_csv('annual_temp_csv2.csv')

# this will make the 'Source' column the index labels
# df = pd.read_csv('annual_temp_csv2.csv', index_col = 'Source')

In [0]:
# have a look at the first bit of the DF
df.head()

## Just like with Series object, can compute mean, std, etc

In [0]:
df['Mean'].mean()

### By default, mean, std etc will skip (ignore) missing values (NaNs)
* Sometimes, its good to do a sanity check if you think there are missing values. 
* Can do this by chosing to NOT skip the NaNs...in which case if they exist you'll get back NaN as the answer!
* Then you know that there are NaNs in the data set. 

In [0]:
df['Mean'].mean(skipna=False)

## Find missing values in your data and deal with them (NaNs)
* Can apply to just one column at a time 
* note that you can call the isna method from the object directly
* To make this work, you index into the data frame where 'Mean' is a Nan 

In [0]:
# isolate just the rows (indicies) where Mean is NaN
df['Mean'].isna()

In [0]:
# now index into df using the true/false sequence from above!
df[df['Mean'].isna()]

#### Or do the opposite, isolate just the rows where Mean is not NaN (i.e. its a real number)

In [0]:
df[df['Mean'].notna()]

### Can deal with NaNs lots of ways...
* Can make a new DF without them
* can assign the mean of all of the data to NaNs

In [0]:
# make a new df, but only keep the non-NaN entries
df2 = df[df['Mean'].notna()]
df2.head()

#### Fill the NaN with the mean of the column!
* Or any other value...just pass it into fillna
* see also 'interpolate' for more functions like this 
* NEED TO ASSIGN output to apply changes..e.g df = df.fillna(....)

In [0]:
print(df['Mean'].mean())
df.fillna(df['Mean'].mean())

## Pull out selected data and remove from DF

In [0]:
df.head()

In [0]:
# or your could self assign df = df[] here to update existing data frame
df2 = df[df['Source'] == 'GISTEMP']
df2.head()

## Grab a range of rows...across a set of years, for example

In [0]:
df2 = df[(df['Year']>1990) & (df['Year']<=2015)]
display(df2)

## Apply several filters at once!
* Be careful here - readability of code is the prime directive...don't write one-liners that are so dense that nobody can understand them!

In [0]:
df2 = df[(df['Source']=='GCAG') & (df['Year']>1990) & (df['Year']<=2015)]
display(df2)

## More on indexing and selection of specific coordinates in a DF

### Row selection - this is a bit more complex as there are many methods
* You can use df.loc to select a row by its label name
* You can use df.iloc to select a row by its integer location (from 0 to length-1 of the axis)
* You can use boolean vectors to select a set of rows that satisfy some condition
* You can slice rows using standard notation e.g. df[1:3] for rows 1-3
* You can also isolate a particular row/column using a combo of column indexing (see above) and standard slicing notation

Contrary to usual slicing conventions, both the start and the stop indices are included when using the DF.LOC option...see below for demo. This makes sense because you're indexing by label name, not by a zero-based integer index. 


In [0]:
df.head()

### this returns the 5th-8th rows beacuse our index labels (row labels) start at 0...

In [0]:
df.loc[4:7]

### Grab a set of non-continguous locations
* just pass in a **list** of the locations that you want!

In [0]:
df.loc[[4,8,100]]

##### Note that indexing by location is **inclusive** of stop value!
* you are not indexing by row number, but by row label! so you get exactly what you ask for (not 0-based indexing in this case). 

In [0]:
df.loc[13:17]

## Use df.loc to find rows that match some criterion (like all rows that contain temperature measurements from the GCAG baseline)

In [0]:
df.loc[df.Source=='GCAG']

## Adding a column is easy and can be done dynamically (on the fly)
* Make a new column of True and False to mark years above/below mean temp deviation

In [0]:
mean_temp = df['Mean'].mean()

print('mean temp:', mean_temp)

# then populate the new column
df['HighLow'] = df['Mean']>mean_temp

df.head()

## If you want to convert values in a column, can be a little tricky...
* Use what you might think is the intuitive way to convert True to 1 and False to 0 in our new column 'HighLow'
* **This throws a weird warning because you're trying to modify the thing that you're using as an index!**

In [0]:
df.HighLow[df.HighLow==True] = 1
df.HighLow[df.HighLow==False] = 0
df.head()

## Solution - use .loc to return the information and then modify it

In [0]:
# reload our df, re-create a new version of our HighLow column

df = pd.read_csv('annual_temp_csv2.csv')

# make our new column again. 
mean_temp = df['Mean'].mean()

print('mean temp:', mean_temp)

# then populate the new column
df['HighLow'] = df['Mean']>mean_temp

df.head()

### like this...using the .loc method ensures that things don't get confused...

In [0]:
df.loc[df['HighLow']==True, 'HighLow'] = 1
df.loc[df['HighLow']==False, 'HighLow'] = 0
df.head()